In [1]:
%run auxiliary.ipynb

In [2]:
all_df = pd.read_excel('all_releases_no_repetitions.xlsx') # Original dataa

In [3]:
results_dict = dict()

In [4]:
k_fold = StratifiedKFold(10, random_state=42)

X_features = ['CBO','CC','LCOM','WMC']

X = pd.DataFrame(all_df.loc[:, X_features])

y = pd.DataFrame(all_df.loc[:, 'will_change'])
X = X.rename(columns={'CBO':'f0', 'CC':'f1', 'LCOM':'f2', 'WMC':'f3'})

In [5]:
X.head()

,f0,f1,f2,f3
0,0,2,0.0,1
1,0,2,0.0,1
2,21,9,0.0,4
3,1,7,0.0,2
4,1,2,0.0,1


In [6]:
parameters_svc_linear = {
        'clf__kernel':['linear'],
    'clf__max_iter':[10000],
    }

parameters_svc_rbf = {
        'clf__kernel':['rbf'],
    }

parameters_dtc_grid = {
    'clf__criterion':['gini'], 
}

parameters_rfc_grid = { 
    'clf__criterion' :['gini'],
}

parameters_ABC_grid = {
    'clf__base_estimator__criterion' : ["gini"],
}


parameters_knn_grid = {'clf__n_neighbors': [5]}


parameters_lr_grid = [{
                        'clf__penalty':['l2'],
                        }]

parameters_GBC = {
    "clf__loss":["deviance"],
    }


param_xgb_grid = {
           'clf__clf__random_state': [42] #ensemble xgboost with multiple seeds may reduce variance
             }

param_lgm_grid = {
    "clf__random_state":[42]
}

# SMOTE

In [7]:
sampler = SMOTE(random_state=42)

## Light GBM

In [8]:
%%time

base_clf = Pipeline([
        ('sampling', sampler),
        ('clf', lbm.LGBMClassifier())
    ])


clf = GridSearchCV(base_clf, param_lgm_grid, scoring='roc_auc', n_jobs=-1)

scores_list = list()

results_dict['Light GBM - SMOTE'] = dict()

for k, (train, test) in enumerate(k_fold.split(X, y)):
    clf.fit(X.iloc[train], y.iloc[train])
    
    print('Fold: ' + str(k + 1))
    
    print('ROC_AUC: ' + str(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test]))))
    print('Estimator and parameters: ' + str(clf.get_params()['estimator']))
    cnf_matrix = confusion_matrix(y.iloc[test], clf.predict(X.iloc[test]))
    cnf_matrix = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]
    print('[Confusion Matrix]: [' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']')
    
    results_dict['Light GBM - SMOTE'][k + 1] = [get_scores(y.iloc[test], clf.predict(X.iloc[test])), 
                                                             str(clf.get_params()['estimator']),
                                                             '[' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']']
    
    scores_list.append(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test])))
    
print('Mean: ' + str(np.mean(scores_list)))
print('Std deviation: ' + str(np.std(scores_list)))

Fold: 1
ROC_AUC: 0.5549291237113402
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', SMOTE(k_neighbors=5, kind='deprecated', m_neighbors='deprecated', n_jobs=1,
   out_step='deprecated', random_state=42, ratio=None,
   sampling_strategy='auto', svm_estimator='deprecated')), ('clf', LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
    ...0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0))])
[Confusion Matrix]: [0.9536082474226805, 0.16]
Fold: 2
ROC_AUC: 0.5978682170542636
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', SMOTE(k_neighbors=5, kind='deprecated', m_neighbors='deprecated', n_jobs=1,
   out_step='deprecated', random_state=42, ratio=None,
   sampling_strategy='auto', svm_estimator='deprecated')), ('clf', LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
    ...0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_fo

In [9]:
with open('base_Over_wo_wfs.pickle', 'wb') as handle:
    pickle.dump(results_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Logistic Regression

In [10]:
%%time

base_clf = Pipeline([
        ('sampling', sampler),
        ('clf', LogisticRegression(random_state=42))
    ])


clf = GridSearchCV(base_clf, parameters_lr_grid, scoring='roc_auc', n_jobs=-1)

scores_list = list()

results_dict['Logistic Regression - SMOTE'] = dict()

for k, (train, test) in enumerate(k_fold.split(X, y)):
    clf.fit(X.iloc[train], y.iloc[train])
    
    print('Fold: ' + str(k + 1))
    
    print('ROC_AUC: ' + str(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test]))))
    print('Estimator and parameters: ' + str(clf.get_params()['estimator']))
    cnf_matrix = confusion_matrix(y.iloc[test], clf.predict(X.iloc[test]))
    cnf_matrix = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]
    print('[Confusion Matrix]: [' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']')
    
    results_dict['Logistic Regression - SMOTE'][k + 1] = [get_scores(y.iloc[test], clf.predict(X.iloc[test])), 
                                                             str(clf.get_params()['estimator']),
                                                             '[' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']']
    
    scores_list.append(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test])))
    
print('Mean: ' + str(np.mean(scores_list)))
print('Std deviation: ' + str(np.std(scores_list)))

Fold: 1
ROC_AUC: 0.6765463917525772
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', SMOTE(k_neighbors=5, kind='deprecated', m_neighbors='deprecated', n_jobs=1,
   out_step='deprecated', random_state=42, ratio=None,
   sampling_strategy='auto', svm_estimator='deprecated')), ('clf', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=42, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))])
[Confusion Matrix]: [0.8530927835051546, 0.5]
Fold: 2
ROC_AUC: 0.759891795865633
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', SMOTE(k_neighbors=5, kind='deprecated', m_neighbors='deprecated', n_jobs=1,
   out_step='deprecated', random_state=42, ratio=None,
   sampling_strategy='auto', svm_estimator='deprecated')), ('clf', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,

In [11]:
with open('base_Over_wo_wfs.pickle', 'wb') as handle:
    pickle.dump(results_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

## SVM - Linear

In [12]:
%%time

base_clf = Pipeline([
        ('sampling', sampler),
        ('clf', svm.SVC(random_state=42))
    ])

clf = GridSearchCV(base_clf, parameters_svc_linear, scoring='roc_auc', n_jobs=-1)

scores_list = list()

results_dict['SVM Linear - SMOTE'] = dict()

for k, (train, test) in enumerate(k_fold.split(X, y)):
    clf.fit(X.iloc[train], y.iloc[train])
    
    print('Fold: ' + str(k + 1))
    
    print('ROC_AUC: ' + str(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test]))))
    print('Estimator and parameters: ' + str(clf.get_params()['estimator']))
    cnf_matrix = confusion_matrix(y.iloc[test], clf.predict(X.iloc[test]))
    cnf_matrix = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]
    print('[Confusion Matrix]: [' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']')
    
    results_dict['SVM Linear - SMOTE'][k + 1] = [get_scores(y.iloc[test], clf.predict(X.iloc[test])), 
                                                             str(clf.get_params()['estimator']),
                                                             '[' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']']
    
    scores_list.append(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test])))
    
print('Mean: ' + str(np.mean(scores_list)))
print('Std deviation: ' + str(np.std(scores_list)))

Fold: 1
ROC_AUC: 0.421875
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', SMOTE(k_neighbors=5, kind='deprecated', m_neighbors='deprecated', n_jobs=1,
   out_step='deprecated', random_state=42, ratio=None,
   sampling_strategy='auto', svm_estimator='deprecated')), ('clf', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=42,
  shrinking=True, tol=0.001, verbose=False))])
[Confusion Matrix]: [0.0, 0.84]
Fold: 2
ROC_AUC: 0.6433301033591731
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', SMOTE(k_neighbors=5, kind='deprecated', m_neighbors='deprecated', n_jobs=1,
   out_step='deprecated', random_state=42, ratio=None,
   sampling_strategy='auto', svm_estimator='deprecated')), ('clf', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  

In [13]:
with open('base_Over_wo_wfs.pickle', 'wb') as handle:
    pickle.dump(results_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

## SVM - RBF

In [14]:
%%time

base_clf = Pipeline([
        ('sampling', sampler),
        ('clf', svm.SVC(random_state=42))
    ])

clf = GridSearchCV(base_clf, parameters_svc_rbf, scoring='roc_auc', n_jobs=-1)

scores_list = list()

results_dict['SVM RBF - SMOTE'] = dict()

for k, (train, test) in enumerate(k_fold.split(X, y)):
    clf.fit(X.iloc[train], y.iloc[train])
    
    print('Fold: ' + str(k + 1))
    
    print('ROC_AUC: ' + str(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test]))))
    print('Estimator and parameters: ' + str(clf.get_params()['estimator']))
    cnf_matrix = confusion_matrix(y.iloc[test], clf.predict(X.iloc[test]))
    cnf_matrix = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]
    print('[Confusion Matrix]: [' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']')
    
    results_dict['SVM RBF - SMOTE'][k + 1] = [get_scores(y.iloc[test], clf.predict(X.iloc[test])), 
                                                             str(clf.get_params()['estimator']),
                                                             '[' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']']
    
    scores_list.append(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test])))
    
print('Mean: ' + str(np.mean(scores_list)))
print('Std deviation: ' + str(np.std(scores_list)))

Fold: 1
ROC_AUC: 0.5792525773195876
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', SMOTE(k_neighbors=5, kind='deprecated', m_neighbors='deprecated', n_jobs=1,
   out_step='deprecated', random_state=42, ratio=None,
   sampling_strategy='auto', svm_estimator='deprecated')), ('clf', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=42,
  shrinking=True, tol=0.001, verbose=False))])
[Confusion Matrix]: [0.7835051546391752, 0.38]
Fold: 2
ROC_AUC: 0.6273821059431525
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', SMOTE(k_neighbors=5, kind='deprecated', m_neighbors='deprecated', n_jobs=1,
   out_step='deprecated', random_state=42, ratio=None,
   sampling_strategy='auto', svm_estimator='deprecated')), ('clf', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, ga

In [15]:
with open('base_Over_wo_wfs.pickle', 'wb') as handle:
    pickle.dump(results_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Decision Tree

In [16]:
%%time

base_clf = Pipeline([
        ('sampling', sampler),
        ('clf', DecisionTreeClassifier(random_state=42))
    ])

clf = GridSearchCV(base_clf, parameters_dtc_grid, scoring='roc_auc', n_jobs=-1)

scores_list = list()

results_dict['Decision Tree - SMOTE'] = dict()

for k, (train, test) in enumerate(k_fold.split(X, y)):
    clf.fit(X.iloc[train], y.iloc[train])
    
    print('Fold: ' + str(k + 1))
    
    print('ROC_AUC: ' + str(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test]))))
    print('Estimator and parameters: ' + str(clf.get_params()['estimator']))
    cnf_matrix = confusion_matrix(y.iloc[test], clf.predict(X.iloc[test]))
    cnf_matrix = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]
    print('[Confusion Matrix]: [' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']')
    
    results_dict['Decision Tree - SMOTE'][k + 1] = [get_scores(y.iloc[test], clf.predict(X.iloc[test])), 
                                                             str(clf.get_params()['estimator']),
                                                             '[' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']']
    
    scores_list.append(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test])))
    
print('Mean: ' + str(np.mean(scores_list)))
print('Std deviation: ' + str(np.std(scores_list)))

Fold: 1
ROC_AUC: 0.517235824742268
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', SMOTE(k_neighbors=5, kind='deprecated', m_neighbors='deprecated', n_jobs=1,
   out_step='deprecated', random_state=42, ratio=None,
   sampling_strategy='auto', svm_estimator='deprecated')), ('clf', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
      ...        min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best'))])
[Confusion Matrix]: [0.9407216494845361, 0.09]
Fold: 2
ROC_AUC: 0.62265826873385
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', SMOTE(k_neighbors=5, kind='deprecated', m_neighbors='deprecated', n_jobs=1,
   out_step='deprecated', random_state=42, ratio=None,
   sampling_strategy='auto', svm_estimator='deprecated')), ('clf', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
      ...        min_weight_fraction_leaf=0.0, presort=False, random_state=42,


In [17]:
with open('base_Over_wo_wfs.pickle', 'wb') as handle:
    pickle.dump(results_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Random Forest

In [18]:
%%time


base_clf = Pipeline([
        ('sampling', sampler),
        ('clf', RandomForestClassifier(random_state=42))
    ])

clf = GridSearchCV(base_clf, parameters_rfc_grid, scoring='roc_auc', n_jobs=-1)

scores_list = list()

results_dict['Random Forest - SMOTE'] = dict()

for k, (train, test) in enumerate(k_fold.split(X, y)):
    clf.fit(X.iloc[train], y.iloc[train])
    
    print('Fold: ' + str(k + 1))
    
    print('ROC_AUC: ' + str(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test]))))
    print('Estimator and parameters: ' + str(clf.get_params()['estimator']))
    cnf_matrix = confusion_matrix(y.iloc[test], clf.predict(X.iloc[test]))
    cnf_matrix = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]
    print('[Confusion Matrix]: [' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']')
    
    results_dict['Random Forest - SMOTE'][k + 1] = [get_scores(y.iloc[test], clf.predict(X.iloc[test])), 
                                                             str(clf.get_params()['estimator']),
                                                             '[' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']']
    
    scores_list.append(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test])))
    
print('Mean: ' + str(np.mean(scores_list)))
print('Std deviation: ' + str(np.std(scores_list)))

Fold: 1
ROC_AUC: 0.5315721649484536
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', SMOTE(k_neighbors=5, kind='deprecated', m_neighbors='deprecated', n_jobs=1,
   out_step='deprecated', random_state=42, ratio=None,
   sampling_strategy='auto', svm_estimator='deprecated')), ('clf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
      ...rs='warn', n_jobs=None,
            oob_score=False, random_state=42, verbose=0, warm_start=False))])
[Confusion Matrix]: [0.9381443298969072, 0.12]
Fold: 2
ROC_AUC: 0.6552002583979328
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', SMOTE(k_neighbors=5, kind='deprecated', m_neighbors='deprecated', n_jobs=1,
   out_step='deprecated', random_state=42, ratio=None,
   sampling_strategy='auto', svm_estimator='deprecated')), ('clf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
      ...rs='warn', n_jobs=None,
            oob_score=False, random_state=4

In [19]:
with open('base_Over_wo_wfs.pickle', 'wb') as handle:
    pickle.dump(results_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

## KNN

In [20]:
%%time

base_clf = Pipeline([
        ('sampling', sampler),
        ('clf', KNeighborsClassifier())
    ])

clf = GridSearchCV(base_clf, parameters_knn_grid, scoring='roc_auc', n_jobs=-1)

scores_list = list()

results_dict['KNN - SMOTE'] = dict()

for k, (train, test) in enumerate(k_fold.split(X, y)):
    clf.fit(X.iloc[train], y.iloc[train])
    
    print('Fold: ' + str(k + 1))
    
    print('ROC_AUC: ' + str(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test]))))
    print('Estimator and parameters: ' + str(clf.get_params()['estimator']))
    cnf_matrix = confusion_matrix(y.iloc[test], clf.predict(X.iloc[test]))
    cnf_matrix = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]
    print('[Confusion Matrix]: [' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']')
    
    results_dict['KNN - SMOTE'][k + 1] = [get_scores(y.iloc[test], clf.predict(X.iloc[test])), 
                                                             str(clf.get_params()['estimator']),
                                                             '[' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']']
    
    scores_list.append(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test])))
    
print('Mean: ' + str(np.mean(scores_list)))
print('Std deviation: ' + str(np.std(scores_list)))

Fold: 1
ROC_AUC: 0.569104381443299
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', SMOTE(k_neighbors=5, kind='deprecated', m_neighbors='deprecated', n_jobs=1,
   out_step='deprecated', random_state=42, ratio=None,
   sampling_strategy='auto', svm_estimator='deprecated')), ('clf', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform'))])
[Confusion Matrix]: [0.7319587628865979, 0.41]
Fold: 2
ROC_AUC: 0.5856750645994832
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', SMOTE(k_neighbors=5, kind='deprecated', m_neighbors='deprecated', n_jobs=1,
   out_step='deprecated', random_state=42, ratio=None,
   sampling_strategy='auto', svm_estimator='deprecated')), ('clf', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform'))])


In [21]:
with open('base_Over_wo_wfs.pickle', 'wb') as handle:
    pickle.dump(results_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Adaboost

In [22]:
%%time

adaboost_base = DecisionTreeClassifier(random_state = 42)

base_clf = Pipeline([
        ('sampling', sampler),
        ('clf', AdaBoostClassifier(random_state=42, base_estimator=adaboost_base))
    ])

clf = GridSearchCV(base_clf, parameters_ABC_grid, scoring='roc_auc', n_jobs=-1)

scores_list = list()

results_dict['Adaboost - SMOTE'] = dict()

for k, (train, test) in enumerate(k_fold.split(X, y)):
    clf.fit(X.iloc[train], y.iloc[train])
    
    print('Fold: ' + str(k + 1))
    
    print('ROC_AUC: ' + str(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test]))))
    print('Estimator and parameters: ' + str(clf.get_params()['estimator']))
    cnf_matrix = confusion_matrix(y.iloc[test], clf.predict(X.iloc[test]))
    cnf_matrix = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]
    print('[Confusion Matrix]: [' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']')
    
    results_dict['Adaboost - SMOTE'][k + 1] = [get_scores(y.iloc[test], clf.predict(X.iloc[test])), 
                                                             str(clf.get_params()['estimator']),
                                                             '[' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']']
    
    scores_list.append(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test])))
    
print('Mean: ' + str(np.mean(scores_list)))
print('Std deviation: ' + str(np.std(scores_list)))

Fold: 1
ROC_AUC: 0.5576675257731959
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', SMOTE(k_neighbors=5, kind='deprecated', m_neighbors='deprecated', n_jobs=1,
   out_step='deprecated', random_state=42, ratio=None,
   sampling_strategy='auto', svm_estimator='deprecated')), ('clf', AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClass...e=42,
            splitter='best'),
          learning_rate=1.0, n_estimators=50, random_state=42))])
[Confusion Matrix]: [0.9278350515463918, 0.19]
Fold: 2
ROC_AUC: 0.5746124031007752
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', SMOTE(k_neighbors=5, kind='deprecated', m_neighbors='deprecated', n_jobs=1,
   out_step='deprecated', random_state=42, ratio=None,
   sampling_strategy='auto', svm_estimator='deprecated')), ('clf', AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClass...e=42,
            splitter='best'),
          learning_rate=1.0, n_

In [23]:
with open('base_Over_wo_wfs.pickle', 'wb') as handle:
    pickle.dump(results_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Gradient Boost

In [24]:
%%time

base_clf = Pipeline([
        ('sampling', sampler),
        ('clf', GradientBoostingClassifier(random_state=42))
    ])


clf = GridSearchCV(base_clf, parameters_GBC, scoring='roc_auc', n_jobs=-1)


scores_list = list()

results_dict['Gradient Boost - SMOTE'] = dict()

for k, (train, test) in enumerate(k_fold.split(X, y)):
    clf.fit(X.iloc[train], y.iloc[train])
    
    print('Fold: ' + str(k + 1))
    
    print('ROC_AUC: ' + str(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test]))))
    print('Estimator and parameters: ' + str(clf.get_params()['estimator']))
    cnf_matrix = confusion_matrix(y.iloc[test], clf.predict(X.iloc[test]))
    cnf_matrix = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]
    print('[Confusion Matrix]: [' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']')
    
    results_dict['Gradient Boost - SMOTE'][k + 1] = [get_scores(y.iloc[test], clf.predict(X.iloc[test])), 
                                                             str(clf.get_params()['estimator']),
                                                             '[' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']']
    
    scores_list.append(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test])))
    
print('Mean: ' + str(np.mean(scores_list)))
print('Std deviation: ' + str(np.std(scores_list)))

Fold: 1
ROC_AUC: 0.6151739690721649
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', SMOTE(k_neighbors=5, kind='deprecated', m_neighbors='deprecated', n_jobs=1,
   out_step='deprecated', random_state=42, ratio=None,
   sampling_strategy='auto', svm_estimator='deprecated')), ('clf', GradientBoostingClassifier(criterion='friedman_mse', init=None,
              lear...    subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False))])
[Confusion Matrix]: [0.8865979381443299, 0.34]
Fold: 2
ROC_AUC: 0.6424014857881137
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', SMOTE(k_neighbors=5, kind='deprecated', m_neighbors='deprecated', n_jobs=1,
   out_step='deprecated', random_state=42, ratio=None,
   sampling_strategy='auto', svm_estimator='deprecated')), ('clf', GradientBoostingClassifier(criterion='friedman_mse', init=None,
              lear...    subsample=1.0, tol=0.0001, validation_fraction=0.1,
           

In [25]:
with open('base_Over_wo_wfs.pickle', 'wb') as handle:
    pickle.dump(results_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

## XGBoost

In [26]:
%%time


base_clf = Pipeline([
        ('sampling', sampler),
        ('clf', xgb.XGBClassifier(random_state=42))
    ])


clf = GridSearchCV(base_clf, param_xgb_grid, scoring='roc_auc', n_jobs=-1)

scores_list = list()

results_dict['XGBoost - SMOTE'] = dict()

for k, (train, test) in enumerate(k_fold.split(X, y)):
    clf.fit(X.iloc[train], y.iloc[train])
    
    print('Fold: ' + str(k + 1))
    
    print('ROC_AUC: ' + str(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test]))))
    print('Estimator and parameters: ' + str(clf.get_params()['estimator']))
    cnf_matrix = confusion_matrix(y.iloc[test], clf.predict(X.iloc[test]))
    cnf_matrix = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]
    print('[Confusion Matrix]: [' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']')
    
    results_dict['XGBoost - SMOTE'][k + 1] = [get_scores(y.iloc[test], clf.predict(X.iloc[test])), 
                                                             str(clf.get_params()['estimator']),
                                                             '[' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']']
    
    scores_list.append(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test])))
    
print('Mean: ' + str(np.mean(scores_list)))
print('Std deviation: ' + str(np.std(scores_list)))

Fold: 1
ROC_AUC: 0.6464239690721649
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', SMOTE(k_neighbors=5, kind='deprecated', m_neighbors='deprecated', n_jobs=1,
   out_step='deprecated', random_state=42, ratio=None,
   sampling_strategy='auto', svm_estimator='deprecated')), ('clf', XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsam...ate=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1))])
[Confusion Matrix]: [0.8865979381443299, 0.41]
Fold: 2
ROC_AUC: 0.6710675064599484
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', SMOTE(k_neighbors=5, kind='deprecated', m_neighbors='deprecated', n_jobs=1,
   out_step='deprecated', random_state=42, ratio=None,
   sampling_strategy='auto', svm_estimator='deprecated')), ('clf', XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsam...ate=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=

In [27]:
with open('base_Over_wo_wfs.pickle', 'wb') as handle:
    pickle.dump(results_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

# ADASYN

In [28]:
sampler = ADASYN(random_state=42)

## Light GBM

In [29]:
%%time

base_clf = Pipeline([
        ('sampling', sampler),
        ('clf', lbm.LGBMClassifier())
    ])


clf = GridSearchCV(base_clf, param_lgm_grid, scoring='roc_auc', n_jobs=-1)

scores_list = list()

results_dict['Light GBM - ADASYN'] = dict()

for k, (train, test) in enumerate(k_fold.split(X, y)):
    clf.fit(X.iloc[train], y.iloc[train])
    
    print('Fold: ' + str(k + 1))
    
    print('ROC_AUC: ' + str(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test]))))
    print('Estimator and parameters: ' + str(clf.get_params()['estimator']))
    cnf_matrix = confusion_matrix(y.iloc[test], clf.predict(X.iloc[test]))
    cnf_matrix = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]
    print('[Confusion Matrix]: [' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']')
    
    results_dict['Light GBM - ADASYN'][k + 1] = [get_scores(y.iloc[test], clf.predict(X.iloc[test])), 
                                                             str(clf.get_params()['estimator']),
                                                             '[' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']']
    
    scores_list.append(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test])))
    
print('Mean: ' + str(np.mean(scores_list)))
print('Std deviation: ' + str(np.std(scores_list)))

Fold: 1
ROC_AUC: 0.5587951030927836
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', ADASYN(n_jobs=1, n_neighbors=5, random_state=42, ratio=None,
    sampling_strategy='auto')), ('clf', LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_c...0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0))])
[Confusion Matrix]: [0.961340206185567, 0.16]
Fold: 2
ROC_AUC: 0.6134932170542636
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', ADASYN(n_jobs=1, n_neighbors=5, random_state=42, ratio=None,
    sampling_strategy='auto')), ('clf', LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_c...0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for

In [30]:
with open('base_Over_wo_wfs.pickle', 'wb') as handle:
    pickle.dump(results_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Logistic Regression

In [31]:
%%time

base_clf = Pipeline([
        ('sampling', sampler),
        ('clf', LogisticRegression(random_state=42))
    ])


clf = GridSearchCV(base_clf, parameters_lr_grid, scoring='roc_auc', n_jobs=-1)

scores_list = list()

results_dict['Logistic Regression - ADASYN'] = dict()

for k, (train, test) in enumerate(k_fold.split(X, y)):
    clf.fit(X.iloc[train], y.iloc[train])
    
    print('Fold: ' + str(k + 1))
    
    print('ROC_AUC: ' + str(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test]))))
    print('Estimator and parameters: ' + str(clf.get_params()['estimator']))
    cnf_matrix = confusion_matrix(y.iloc[test], clf.predict(X.iloc[test]))
    cnf_matrix = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]
    print('[Confusion Matrix]: [' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']')
    
    results_dict['Logistic Regression - ADASYN'][k + 1] = [get_scores(y.iloc[test], clf.predict(X.iloc[test])), 
                                                             str(clf.get_params()['estimator']),
                                                             '[' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']']
    
    scores_list.append(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test])))
    
print('Mean: ' + str(np.mean(scores_list)))
print('Std deviation: ' + str(np.std(scores_list)))

Fold: 1
ROC_AUC: 0.6557667525773196
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', ADASYN(n_jobs=1, n_neighbors=5, random_state=42, ratio=None,
    sampling_strategy='auto')), ('clf', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=42, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))])
[Confusion Matrix]: [0.8427835051546392, 0.47]
Fold: 2
ROC_AUC: 0.7456799095607235
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', ADASYN(n_jobs=1, n_neighbors=5, random_state=42, ratio=None,
    sampling_strategy='auto')), ('clf', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=42, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))])
[Co

In [32]:
with open('base_Over_wo_wfs.pickle', 'wb') as handle:
    pickle.dump(results_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

## SVM - Linear

In [33]:
%%time

base_clf = Pipeline([
        ('sampling', sampler),
        ('clf', svm.SVC(random_state=42))
    ])

clf = GridSearchCV(base_clf, parameters_svc_linear, scoring='roc_auc', n_jobs=-1)

scores_list = list()

results_dict['SVM Linear - ADASYN'] = dict()

for k, (train, test) in enumerate(k_fold.split(X, y)):
    clf.fit(X.iloc[train], y.iloc[train])
    
    print('Fold: ' + str(k + 1))
    
    print('ROC_AUC: ' + str(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test]))))
    print('Estimator and parameters: ' + str(clf.get_params()['estimator']))
    cnf_matrix = confusion_matrix(y.iloc[test], clf.predict(X.iloc[test]))
    cnf_matrix = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]
    print('[Confusion Matrix]: [' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']')
    
    results_dict['SVM Linear - ADASYN'][k + 1] = [get_scores(y.iloc[test], clf.predict(X.iloc[test])), 
                                                             str(clf.get_params()['estimator']),
                                                             '[' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']']
    
    scores_list.append(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test])))
    
print('Mean: ' + str(np.mean(scores_list)))
print('Std deviation: ' + str(np.std(scores_list)))

Fold: 1
ROC_AUC: 0.5873067010309279
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', ADASYN(n_jobs=1, n_neighbors=5, random_state=42, ratio=None,
    sampling_strategy='auto')), ('clf', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=42,
  shrinking=True, tol=0.001, verbose=False))])
[Confusion Matrix]: [0.9871134020618557, 0.19]
Fold: 2
ROC_AUC: 0.30050872093023256
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', ADASYN(n_jobs=1, n_neighbors=5, random_state=42, ratio=None,
    sampling_strategy='auto')), ('clf', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=42,
  shrinking=True, tol=0.001, verbose=False))])
[Confusion Matrix]: [0.06976744186046512, 0.53]
Fold: 3
RO

In [34]:
with open('base_Over_wo_wfs.pickle', 'wb') as handle:
    pickle.dump(results_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

## SVM - RBF

In [35]:
%%time

base_clf = Pipeline([
        ('sampling', sampler),
        ('clf', svm.SVC(random_state=42))
    ])

clf = GridSearchCV(base_clf, parameters_svc_rbf, scoring='roc_auc', n_jobs=-1)

scores_list = list()

results_dict['SVM RBF - ADASYN'] = dict()

for k, (train, test) in enumerate(k_fold.split(X, y)):
    clf.fit(X.iloc[train], y.iloc[train])
    
    print('Fold: ' + str(k + 1))
    
    print('ROC_AUC: ' + str(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test]))))
    print('Estimator and parameters: ' + str(clf.get_params()['estimator']))
    cnf_matrix = confusion_matrix(y.iloc[test], clf.predict(X.iloc[test]))
    cnf_matrix = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]
    print('[Confusion Matrix]: [' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']')
    
    results_dict['SVM RBF - ADASYN'][k + 1] = [get_scores(y.iloc[test], clf.predict(X.iloc[test])), 
                                                             str(clf.get_params()['estimator']),
                                                             '[' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']']
    
    scores_list.append(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test])))
    
print('Mean: ' + str(np.mean(scores_list)))
print('Std deviation: ' + str(np.std(scores_list)))

Fold: 1
ROC_AUC: 0.5729703608247423
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', ADASYN(n_jobs=1, n_neighbors=5, random_state=42, ratio=None,
    sampling_strategy='auto')), ('clf', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=42,
  shrinking=True, tol=0.001, verbose=False))])
[Confusion Matrix]: [0.7396907216494846, 0.41]
Fold: 2
ROC_AUC: 0.653343023255814
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', ADASYN(n_jobs=1, n_neighbors=5, random_state=42, ratio=None,
    sampling_strategy='auto')), ('clf', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=42,
  shrinking=True, tol=0.001, verbose=False))])
[Confusion Matrix]: [0.7441860465116279, 0.56]
Fold: 3
ROC_A

In [36]:
with open('base_Over_wo_wfs.pickle', 'wb') as handle:
    pickle.dump(results_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Decision Tree

In [37]:
%%time

base_clf = Pipeline([
        ('sampling', sampler),
        ('clf', DecisionTreeClassifier(random_state=42))
    ])

clf = GridSearchCV(base_clf, parameters_dtc_grid, scoring='roc_auc', n_jobs=-1)

scores_list = list()

results_dict['Decision Tree - ADASYN'] = dict()

for k, (train, test) in enumerate(k_fold.split(X, y)):
    clf.fit(X.iloc[train], y.iloc[train])
    
    print('Fold: ' + str(k + 1))
    
    print('ROC_AUC: ' + str(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test]))))
    print('Estimator and parameters: ' + str(clf.get_params()['estimator']))
    cnf_matrix = confusion_matrix(y.iloc[test], clf.predict(X.iloc[test]))
    cnf_matrix = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]
    print('[Confusion Matrix]: [' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']')
    
    results_dict['Decision Tree - ADASYN'][k + 1] = [get_scores(y.iloc[test], clf.predict(X.iloc[test])), 
                                                             str(clf.get_params()['estimator']),
                                                             '[' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']']
    
    scores_list.append(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test])))
    
print('Mean: ' + str(np.mean(scores_list)))
print('Std deviation: ' + str(np.std(scores_list)))

Fold: 1
ROC_AUC: 0.5236791237113402
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', ADASYN(n_jobs=1, n_neighbors=5, random_state=42, ratio=None,
    sampling_strategy='auto')), ('clf', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best'))])
[Confusion Matrix]: [0.9536082474226805, 0.09]
Fold: 2
ROC_AUC: 0.5939922480620156
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', ADASYN(n_jobs=1, n_neighbors=5, random_state=42, ratio=None,
    sampling_strategy='auto')), ('clf', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_sp

In [38]:
with open('base_Over_wo_wfs.pickle', 'wb') as handle:
    pickle.dump(results_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Random Forest

In [39]:
%%time


base_clf = Pipeline([
        ('sampling', sampler),
        ('clf', RandomForestClassifier(random_state=42))
    ])

clf = GridSearchCV(base_clf, parameters_rfc_grid, scoring='roc_auc', n_jobs=-1)

scores_list = list()

results_dict['Random Forest - ADASYN'] = dict()

for k, (train, test) in enumerate(k_fold.split(X, y)):
    clf.fit(X.iloc[train], y.iloc[train])
    
    print('Fold: ' + str(k + 1))
    
    print('ROC_AUC: ' + str(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test]))))
    print('Estimator and parameters: ' + str(clf.get_params()['estimator']))
    cnf_matrix = confusion_matrix(y.iloc[test], clf.predict(X.iloc[test]))
    cnf_matrix = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]
    print('[Confusion Matrix]: [' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']')
    
    results_dict['Random Forest - ADASYN'][k + 1] = [get_scores(y.iloc[test], clf.predict(X.iloc[test])), 
                                                             str(clf.get_params()['estimator']),
                                                             '[' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']']
    
    scores_list.append(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test])))
    
print('Mean: ' + str(np.mean(scores_list)))
print('Std deviation: ' + str(np.std(scores_list)))

Fold: 1
ROC_AUC: 0.5017719072164949
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', ADASYN(n_jobs=1, n_neighbors=5, random_state=42, ratio=None,
    sampling_strategy='auto')), ('clf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease...rs='warn', n_jobs=None,
            oob_score=False, random_state=42, verbose=0, warm_start=False))])
[Confusion Matrix]: [0.9097938144329897, 0.09]
Fold: 2
ROC_AUC: 0.6395752583979328
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', ADASYN(n_jobs=1, n_neighbors=5, random_state=42, ratio=None,
    sampling_strategy='auto')), ('clf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease...rs='warn', n_jobs=None,
            oob_score=False, random_state=4

In [40]:
with open('base_Over_wo_wfs.pickle', 'wb') as handle:
    pickle.dump(results_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

## KNN

In [41]:
%%time

base_clf = Pipeline([
        ('sampling', sampler),
        ('clf', KNeighborsClassifier())
    ])

clf = GridSearchCV(base_clf, parameters_knn_grid, scoring='roc_auc', n_jobs=-1)

scores_list = list()

results_dict['KNN - ADASYN'] = dict()

for k, (train, test) in enumerate(k_fold.split(X, y)):
    clf.fit(X.iloc[train], y.iloc[train])
    
    print('Fold: ' + str(k + 1))
    
    print('ROC_AUC: ' + str(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test]))))
    print('Estimator and parameters: ' + str(clf.get_params()['estimator']))
    cnf_matrix = confusion_matrix(y.iloc[test], clf.predict(X.iloc[test]))
    cnf_matrix = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]
    print('[Confusion Matrix]: [' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']')
    
    results_dict['KNN - ADASYN'][k + 1] = [get_scores(y.iloc[test], clf.predict(X.iloc[test])), 
                                                             str(clf.get_params()['estimator']),
                                                             '[' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']']
    
    scores_list.append(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test])))
    
print('Mean: ' + str(np.mean(scores_list)))
print('Std deviation: ' + str(np.std(scores_list)))

Fold: 1
ROC_AUC: 0.6092139175257733
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', ADASYN(n_jobs=1, n_neighbors=5, random_state=42, ratio=None,
    sampling_strategy='auto')), ('clf', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform'))])
[Confusion Matrix]: [0.7809278350515464, 0.44]
Fold: 2
ROC_AUC: 0.6375968992248062
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', ADASYN(n_jobs=1, n_neighbors=5, random_state=42, ratio=None,
    sampling_strategy='auto')), ('clf', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform'))])
[Confusion Matrix]: [0.7751937984496124, 0.5]
Fold: 3
ROC_AUC: 0.6056514128532133
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', ADASYN(n_jobs=1, n_neighbors=5, random

In [42]:
with open('base_Over_wo_wfs.pickle', 'wb') as handle:
    pickle.dump(results_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Adaboost

In [43]:
%%time

adaboost_base = DecisionTreeClassifier(random_state = 42)

base_clf = Pipeline([
        ('sampling', sampler),
        ('clf', AdaBoostClassifier(random_state=42, base_estimator=adaboost_base))
    ])

clf = GridSearchCV(base_clf, parameters_ABC_grid, scoring='roc_auc', n_jobs=-1)

scores_list = list()

results_dict['Adaboost - ADASYN'] = dict()

for k, (train, test) in enumerate(k_fold.split(X, y)):
    clf.fit(X.iloc[train], y.iloc[train])
    
    print('Fold: ' + str(k + 1))
    
    print('ROC_AUC: ' + str(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test]))))
    print('Estimator and parameters: ' + str(clf.get_params()['estimator']))
    cnf_matrix = confusion_matrix(y.iloc[test], clf.predict(X.iloc[test]))
    cnf_matrix = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]
    print('[Confusion Matrix]: [' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']')
    
    results_dict['Adaboost - ADASYN'][k + 1] = [get_scores(y.iloc[test], clf.predict(X.iloc[test])), 
                                                             str(clf.get_params()['estimator']),
                                                             '[' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']']
    
    scores_list.append(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test])))
    
print('Mean: ' + str(np.mean(scores_list)))
print('Std deviation: ' + str(np.std(scores_list)))

Fold: 1
ROC_AUC: 0.5422036082474226
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', ADASYN(n_jobs=1, n_neighbors=5, random_state=42, ratio=None,
    sampling_strategy='auto')), ('clf', AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_lea...e=42,
            splitter='best'),
          learning_rate=1.0, n_estimators=50, random_state=42))])
[Confusion Matrix]: [0.8969072164948454, 0.19]
Fold: 2
ROC_AUC: 0.614906330749354
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', ADASYN(n_jobs=1, n_neighbors=5, random_state=42, ratio=None,
    sampling_strategy='auto')), ('clf', AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_lea...e=42,
            splitter='best'),
          learning_rate=1.0, n_e

In [44]:
with open('base_Over_wo_wfs.pickle', 'wb') as handle:
    pickle.dump(results_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Gradient Boost

In [45]:
%%time

base_clf = Pipeline([
        ('sampling', sampler),
        ('clf', GradientBoostingClassifier(random_state=42))
    ])


clf = GridSearchCV(base_clf, parameters_GBC, scoring='roc_auc', n_jobs=-1)


scores_list = list()

results_dict['Gradient Boost - ADASYN'] = dict()

for k, (train, test) in enumerate(k_fold.split(X, y)):
    clf.fit(X.iloc[train], y.iloc[train])
    
    print('Fold: ' + str(k + 1))
    
    print('ROC_AUC: ' + str(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test]))))
    print('Estimator and parameters: ' + str(clf.get_params()['estimator']))
    cnf_matrix = confusion_matrix(y.iloc[test], clf.predict(X.iloc[test]))
    cnf_matrix = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]
    print('[Confusion Matrix]: [' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']')
    
    results_dict['Gradient Boost - ADASYN'][k + 1] = [get_scores(y.iloc[test], clf.predict(X.iloc[test])), 
                                                             str(clf.get_params()['estimator']),
                                                             '[' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']']
    
    scores_list.append(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test])))
    
print('Mean: ' + str(np.mean(scores_list)))
print('Std deviation: ' + str(np.std(scores_list)))

Fold: 1
ROC_AUC: 0.6723582474226805
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', ADASYN(n_jobs=1, n_neighbors=5, random_state=42, ratio=None,
    sampling_strategy='auto')), ('clf', GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None...    subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False))])
[Confusion Matrix]: [0.9072164948453608, 0.44]
Fold: 2
ROC_AUC: 0.656734496124031
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', ADASYN(n_jobs=1, n_neighbors=5, random_state=42, ratio=None,
    sampling_strategy='auto')), ('clf', GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None...    subsample=1.0, tol=0.0001, validation_fraction=0.1,
            

In [46]:
with open('base_Over_wo_wfs.pickle', 'wb') as handle:
    pickle.dump(results_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

## XGBoost

In [47]:
%%time


base_clf = Pipeline([
        ('sampling', sampler),
        ('clf', xgb.XGBClassifier(random_state=42))
    ])


clf = GridSearchCV(base_clf, param_xgb_grid, scoring='roc_auc', n_jobs=-1)

scores_list = list()

results_dict['XGBoost - ADASYN'] = dict()

for k, (train, test) in enumerate(k_fold.split(X, y)):
    clf.fit(X.iloc[train], y.iloc[train])
    
    print('Fold: ' + str(k + 1))
    
    print('ROC_AUC: ' + str(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test]))))
    print('Estimator and parameters: ' + str(clf.get_params()['estimator']))
    cnf_matrix = confusion_matrix(y.iloc[test], clf.predict(X.iloc[test]))
    cnf_matrix = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]
    print('[Confusion Matrix]: [' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']')
    
    results_dict['XGBoost - ADASYN'][k + 1] = [get_scores(y.iloc[test], clf.predict(X.iloc[test])), 
                                                             str(clf.get_params()['estimator']),
                                                             '[' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']']
    
    scores_list.append(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test])))
    
print('Mean: ' + str(np.mean(scores_list)))
print('Std deviation: ' + str(np.std(scores_list)))

Fold: 1
ROC_AUC: 0.6359536082474228
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', ADASYN(n_jobs=1, n_neighbors=5, random_state=42, ratio=None,
    sampling_strategy='auto')), ('clf', XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=...ate=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1))])
[Confusion Matrix]: [0.8969072164948454, 0.38]
Fold: 2
ROC_AUC: 0.672359496124031
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', ADASYN(n_jobs=1, n_neighbors=5, random_state=42, ratio=None,
    sampling_strategy='auto')), ('clf', XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=...ate=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=N

In [48]:
with open('base_Over_wo_wfs.pickle', 'wb') as handle:
    pickle.dump(results_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Random OverSampler

In [49]:
sampler = RandomOverSampler(random_state=42)

## Light GBM

In [50]:
%%time

base_clf = Pipeline([
        ('sampling', sampler),
        ('clf', lbm.LGBMClassifier())
    ])


clf = GridSearchCV(base_clf, param_lgm_grid, scoring='roc_auc', n_jobs=-1)

scores_list = list()

results_dict['Light GBM - RandomOverSampler'] = dict()

for k, (train, test) in enumerate(k_fold.split(X, y)):
    clf.fit(X.iloc[train], y.iloc[train])
    
    print('Fold: ' + str(k + 1))
    
    print('ROC_AUC: ' + str(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test]))))
    print('Estimator and parameters: ' + str(clf.get_params()['estimator']))
    cnf_matrix = confusion_matrix(y.iloc[test], clf.predict(X.iloc[test]))
    cnf_matrix = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]
    print('[Confusion Matrix]: [' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']')
    
    results_dict['Light GBM - RandomOverSampler'][k + 1] = [get_scores(y.iloc[test], clf.predict(X.iloc[test])), 
                                                             str(clf.get_params()['estimator']),
                                                             '[' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']']
    
    scores_list.append(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test])))
    
print('Mean: ' + str(np.mean(scores_list)))
print('Std deviation: ' + str(np.std(scores_list)))

Fold: 1
ROC_AUC: 0.5734536082474226
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', RandomOverSampler(random_state=42, ratio=None, return_indices=False,
         sampling_strategy='auto')), ('clf', LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samp...0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0))])
[Confusion Matrix]: [0.8969072164948454, 0.25]
Fold: 2
ROC_AUC: 0.5736837855297158
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', RandomOverSampler(random_state=42, ratio=None, return_indices=False,
         sampling_strategy='auto')), ('clf', LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samp...0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_fo

In [51]:
with open('base_Over_wo_wfs.pickle', 'wb') as handle:
    pickle.dump(results_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Logistic Regression

In [52]:
%%time

base_clf = Pipeline([
        ('sampling', sampler),
        ('clf', LogisticRegression(random_state=42))
    ])


clf = GridSearchCV(base_clf, parameters_lr_grid, scoring='roc_auc', n_jobs=-1)

scores_list = list()

results_dict['Logistic Regression - RandomOverSampler'] = dict()

for k, (train, test) in enumerate(k_fold.split(X, y)):
    clf.fit(X.iloc[train], y.iloc[train])
    
    print('Fold: ' + str(k + 1))
    
    print('ROC_AUC: ' + str(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test]))))
    print('Estimator and parameters: ' + str(clf.get_params()['estimator']))
    cnf_matrix = confusion_matrix(y.iloc[test], clf.predict(X.iloc[test]))
    cnf_matrix = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]
    print('[Confusion Matrix]: [' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']')
    
    results_dict['Logistic Regression - RandomOverSampler'][k + 1] = [get_scores(y.iloc[test], clf.predict(X.iloc[test])), 
                                                             str(clf.get_params()['estimator']),
                                                             '[' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']']
    
    scores_list.append(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test])))
    
print('Mean: ' + str(np.mean(scores_list)))
print('Std deviation: ' + str(np.std(scores_list)))

Fold: 1
ROC_AUC: 0.672680412371134
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', RandomOverSampler(random_state=42, ratio=None, return_indices=False,
         sampling_strategy='auto')), ('clf', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=42, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))])
[Confusion Matrix]: [0.845360824742268, 0.5]
Fold: 2
ROC_AUC: 0.7494347545219638
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', RandomOverSampler(random_state=42, ratio=None, return_indices=False,
         sampling_strategy='auto')), ('clf', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=42, solver='warn',
          tol=0.0001, verbose=0, w

In [53]:
with open('base_Over_wo_wfs.pickle', 'wb') as handle:
    pickle.dump(results_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

## SVM - Linear

In [54]:
%%time

base_clf = Pipeline([
        ('sampling', sampler),
        ('clf', svm.SVC(random_state=42))
    ])

clf = GridSearchCV(base_clf, parameters_svc_linear, scoring='roc_auc', n_jobs=-1)

scores_list = list()

results_dict['SVM Linear - RandomOverSampler'] = dict()

for k, (train, test) in enumerate(k_fold.split(X, y)):
    clf.fit(X.iloc[train], y.iloc[train])
    
    print('Fold: ' + str(k + 1))
    
    print('ROC_AUC: ' + str(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test]))))
    print('Estimator and parameters: ' + str(clf.get_params()['estimator']))
    cnf_matrix = confusion_matrix(y.iloc[test], clf.predict(X.iloc[test]))
    cnf_matrix = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]
    print('[Confusion Matrix]: [' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']')
    
    results_dict['SVM Linear - RandomOverSampler'][k + 1] = [get_scores(y.iloc[test], clf.predict(X.iloc[test])), 
                                                             str(clf.get_params()['estimator']),
                                                             '[' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']']
    
    scores_list.append(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test])))
    
print('Mean: ' + str(np.mean(scores_list)))
print('Std deviation: ' + str(np.std(scores_list)))

Fold: 1
ROC_AUC: 0.40882731958762886
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', RandomOverSampler(random_state=42, ratio=None, return_indices=False,
         sampling_strategy='auto')), ('clf', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=42,
  shrinking=True, tol=0.001, verbose=False))])
[Confusion Matrix]: [0.005154639175257732, 0.81]
Fold: 2
ROC_AUC: 0.3566698966408269
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', RandomOverSampler(random_state=42, ratio=None, return_indices=False,
         sampling_strategy='auto')), ('clf', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=42,
  shrinking=True, tol=0.001, verbose=False))])
[Confusion Matrix]: [0.0258397

In [55]:
with open('base_Over_wo_wfs.pickle', 'wb') as handle:
    pickle.dump(results_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

## SVM - RBF

In [56]:
%%time

base_clf = Pipeline([
        ('sampling', sampler),
        ('clf', svm.SVC(random_state=42))
    ])

clf = GridSearchCV(base_clf, parameters_svc_rbf, scoring='roc_auc', n_jobs=-1)

scores_list = list()

results_dict['SVM RBF - RandomOverSampler'] = dict()

for k, (train, test) in enumerate(k_fold.split(X, y)):
    clf.fit(X.iloc[train], y.iloc[train])
    
    print('Fold: ' + str(k + 1))
    
    print('ROC_AUC: ' + str(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test]))))
    print('Estimator and parameters: ' + str(clf.get_params()['estimator']))
    cnf_matrix = confusion_matrix(y.iloc[test], clf.predict(X.iloc[test]))
    cnf_matrix = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]
    print('[Confusion Matrix]: [' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']')
    
    results_dict['SVM RBF - RandomOverSampler'][k + 1] = [get_scores(y.iloc[test], clf.predict(X.iloc[test])), 
                                                             str(clf.get_params()['estimator']),
                                                             '[' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']']
    
    scores_list.append(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test])))
    
print('Mean: ' + str(np.mean(scores_list)))
print('Std deviation: ' + str(np.std(scores_list)))

Fold: 1
ROC_AUC: 0.5032216494845361
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', RandomOverSampler(random_state=42, ratio=None, return_indices=False,
         sampling_strategy='auto')), ('clf', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=42,
  shrinking=True, tol=0.001, verbose=False))])
[Confusion Matrix]: [0.8814432989690721, 0.12]
Fold: 2
ROC_AUC: 0.44339470284237725
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', RandomOverSampler(random_state=42, ratio=None, return_indices=False,
         sampling_strategy='auto')), ('clf', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=42,
  shrinking=True, tol=0.001, verbose=False))])
[Confusion Matrix]: [0.824289405

In [57]:
with open('base_Over_wo_wfs.pickle', 'wb') as handle:
    pickle.dump(results_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Decision Tree

In [58]:
%%time

base_clf = Pipeline([
        ('sampling', sampler),
        ('clf', DecisionTreeClassifier(random_state=42))
    ])

clf = GridSearchCV(base_clf, parameters_dtc_grid, scoring='roc_auc', n_jobs=-1)

scores_list = list()

results_dict['Decision Tree - RandomOverSampler'] = dict()

for k, (train, test) in enumerate(k_fold.split(X, y)):
    clf.fit(X.iloc[train], y.iloc[train])
    
    print('Fold: ' + str(k + 1))
    
    print('ROC_AUC: ' + str(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test]))))
    print('Estimator and parameters: ' + str(clf.get_params()['estimator']))
    cnf_matrix = confusion_matrix(y.iloc[test], clf.predict(X.iloc[test]))
    cnf_matrix = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]
    print('[Confusion Matrix]: [' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']')
    
    results_dict['Decision Tree - RandomOverSampler'][k + 1] = [get_scores(y.iloc[test], clf.predict(X.iloc[test])), 
                                                             str(clf.get_params()['estimator']),
                                                             '[' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']']
    
    scores_list.append(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test])))
    
print('Mean: ' + str(np.mean(scores_list)))
print('Std deviation: ' + str(np.std(scores_list)))

Fold: 1
ROC_AUC: 0.49710051546391754
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', RandomOverSampler(random_state=42, ratio=None, return_indices=False,
         sampling_strategy='auto')), ('clf', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best'))])
[Confusion Matrix]: [0.8067010309278351, 0.19]
Fold: 2
ROC_AUC: 0.5266876614987079
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', RandomOverSampler(random_state=42, ratio=None, return_indices=False,
         sampling_strategy='auto')), ('clf', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_de

In [59]:
with open('base_Over_wo_wfs.pickle', 'wb') as handle:
    pickle.dump(results_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Random Forest

In [60]:
%%time

base_clf = Pipeline([
        ('sampling', sampler),
        ('clf', RandomForestClassifier(random_state=42))
    ])

clf = GridSearchCV(base_clf, parameters_rfc_grid, scoring='roc_auc', n_jobs=-1)

scores_list = list()

results_dict['Random Forest - RandomOverSampler'] = dict()

for k, (train, test) in enumerate(k_fold.split(X, y)):
    clf.fit(X.iloc[train], y.iloc[train])
    
    print('Fold: ' + str(k + 1))
    
    print('ROC_AUC: ' + str(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test]))))
    print('Estimator and parameters: ' + str(clf.get_params()['estimator']))
    cnf_matrix = confusion_matrix(y.iloc[test], clf.predict(X.iloc[test]))
    cnf_matrix = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]
    print('[Confusion Matrix]: [' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']')
    
    results_dict['Random Forest - RandomOverSampler'][k + 1] = [get_scores(y.iloc[test], clf.predict(X.iloc[test])), 
                                                             str(clf.get_params()['estimator']),
                                                             '[' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']']
    
    scores_list.append(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test])))
    
print('Mean: ' + str(np.mean(scores_list)))
print('Std deviation: ' + str(np.std(scores_list)))

Fold: 1
ROC_AUC: 0.5191688144329897
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', RandomOverSampler(random_state=42, ratio=None, return_indices=False,
         sampling_strategy='auto')), ('clf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impu...rs='warn', n_jobs=None,
            oob_score=False, random_state=42, verbose=0, warm_start=False))])
[Confusion Matrix]: [0.8195876288659794, 0.22]
Fold: 2
ROC_AUC: 0.5136466408268734
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', RandomOverSampler(random_state=42, ratio=None, return_indices=False,
         sampling_strategy='auto')), ('clf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impu...rs='warn', n_jobs=None,
            oob_score=False, random_state=4

In [61]:
with open('base_Over_wo_wfs.pickle', 'wb') as handle:
    pickle.dump(results_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

## KNN

In [62]:
%%time

base_clf = Pipeline([
        ('sampling', sampler),
        ('clf', KNeighborsClassifier())
    ])

clf = GridSearchCV(base_clf, parameters_knn_grid, scoring='roc_auc', n_jobs=-1)

scores_list = list()

results_dict['KNN - RandomOverSampler'] = dict()

for k, (train, test) in enumerate(k_fold.split(X, y)):
    clf.fit(X.iloc[train], y.iloc[train])
    
    print('Fold: ' + str(k + 1))
    
    print('ROC_AUC: ' + str(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test]))))
    print('Estimator and parameters: ' + str(clf.get_params()['estimator']))
    cnf_matrix = confusion_matrix(y.iloc[test], clf.predict(X.iloc[test]))
    cnf_matrix = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]
    print('[Confusion Matrix]: [' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']')
    
    results_dict['KNN - RandomOverSampler'][k + 1] = [get_scores(y.iloc[test], clf.predict(X.iloc[test])), 
                                                             str(clf.get_params()['estimator']),
                                                             '[' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']']
    
    scores_list.append(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test])))
    
print('Mean: ' + str(np.mean(scores_list)))
print('Std deviation: ' + str(np.std(scores_list)))

Fold: 1
ROC_AUC: 0.6222615979381444
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', RandomOverSampler(random_state=42, ratio=None, return_indices=False,
         sampling_strategy='auto')), ('clf', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform'))])
[Confusion Matrix]: [0.7757731958762887, 0.47]
Fold: 2
ROC_AUC: 0.5763888888888888
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', RandomOverSampler(random_state=42, ratio=None, return_indices=False,
         sampling_strategy='auto')), ('clf', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform'))])
[Confusion Matrix]: [0.7777777777777778, 0.38]
Fold: 3
ROC_AUC: 0.5527215137117614
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', RandomOverS

In [63]:
with open('base_Over_wo_wfs.pickle', 'wb') as handle:
    pickle.dump(results_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Adaboost

In [64]:
%%time

adaboost_base = DecisionTreeClassifier(random_state = 42)

base_clf = Pipeline([
        ('sampling', sampler),
        ('clf', AdaBoostClassifier(random_state=42, base_estimator=adaboost_base))
    ])

clf = GridSearchCV(base_clf, parameters_ABC_grid, scoring='roc_auc', n_jobs=-1)

scores_list = list()

results_dict['Adaboost - RandomOverSampler'] = dict()

for k, (train, test) in enumerate(k_fold.split(X, y)):
    clf.fit(X.iloc[train], y.iloc[train])
    
    print('Fold: ' + str(k + 1))
    
    print('ROC_AUC: ' + str(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test]))))
    print('Estimator and parameters: ' + str(clf.get_params()['estimator']))
    cnf_matrix = confusion_matrix(y.iloc[test], clf.predict(X.iloc[test]))
    cnf_matrix = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]
    print('[Confusion Matrix]: [' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']')
    
    results_dict['Adaboost - RandomOverSampler'][k + 1] = [get_scores(y.iloc[test], clf.predict(X.iloc[test])), 
                                                             str(clf.get_params()['estimator']),
                                                             '[' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']']
    
    scores_list.append(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test])))
    
print('Mean: ' + str(np.mean(scores_list)))
print('Std deviation: ' + str(np.std(scores_list)))

Fold: 1
ROC_AUC: 0.4526417525773196
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', RandomOverSampler(random_state=42, ratio=None, return_indices=False,
         sampling_strategy='auto')), ('clf', AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=...e=42,
            splitter='best'),
          learning_rate=1.0, n_estimators=50, random_state=42))])
[Confusion Matrix]: [0.8427835051546392, 0.06]
Fold: 2
ROC_AUC: 0.46148255813953487
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', RandomOverSampler(random_state=42, ratio=None, return_indices=False,
         sampling_strategy='auto')), ('clf', AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=...e=42,
            splitter='best'),
          learning_rate=1.0, n

In [65]:
with open('base_Over_wo_wfs.pickle', 'wb') as handle:
    pickle.dump(results_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Gradient Boost

In [66]:
%%time

base_clf = Pipeline([
        ('sampling', sampler),
        ('clf', GradientBoostingClassifier(random_state=42))
    ])


clf = GridSearchCV(base_clf, parameters_GBC, scoring='roc_auc', n_jobs=-1)


scores_list = list()

results_dict['Gradient Boost - RandomOverSampler'] = dict()

for k, (train, test) in enumerate(k_fold.split(X, y)):
    clf.fit(X.iloc[train], y.iloc[train])
    
    print('Fold: ' + str(k + 1))
    
    print('ROC_AUC: ' + str(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test]))))
    print('Estimator and parameters: ' + str(clf.get_params()['estimator']))
    cnf_matrix = confusion_matrix(y.iloc[test], clf.predict(X.iloc[test]))
    cnf_matrix = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]
    print('[Confusion Matrix]: [' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']')
    
    results_dict['Gradient Boost - RandomOverSampler'][k + 1] = [get_scores(y.iloc[test], clf.predict(X.iloc[test])), 
                                                             str(clf.get_params()['estimator']),
                                                             '[' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']']
    
    scores_list.append(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test])))
    
print('Mean: ' + str(np.mean(scores_list)))
print('Std deviation: ' + str(np.std(scores_list)))

Fold: 1
ROC_AUC: 0.6728414948453608
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', RandomOverSampler(random_state=42, ratio=None, return_indices=False,
         sampling_strategy='auto')), ('clf', GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_le...    subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False))])
[Confusion Matrix]: [0.8144329896907216, 0.53]
Fold: 2
ROC_AUC: 0.6729651162790697
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', RandomOverSampler(random_state=42, ratio=None, return_indices=False,
         sampling_strategy='auto')), ('clf', GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_le...    subsample=1.0, tol=0.0001, validation_fraction=0.1,
           

In [67]:
with open('base_Over_wo_wfs.pickle', 'wb') as handle:
    pickle.dump(results_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

## XGBoost

In [68]:
%%time


base_clf = Pipeline([
        ('sampling', sampler),
        ('clf', xgb.XGBClassifier(random_state=42))
    ])


clf = GridSearchCV(base_clf, param_xgb_grid, scoring='roc_auc', n_jobs=-1)

scores_list = list()

results_dict['XGBoost - RandomOverSampler'] = dict()

for k, (train, test) in enumerate(k_fold.split(X, y)):
    clf.fit(X.iloc[train], y.iloc[train])
    
    print('Fold: ' + str(k + 1))
    
    print('ROC_AUC: ' + str(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test]))))
    print('Estimator and parameters: ' + str(clf.get_params()['estimator']))
    cnf_matrix = confusion_matrix(y.iloc[test], clf.predict(X.iloc[test]))
    cnf_matrix = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]
    print('[Confusion Matrix]: [' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']')
    
    results_dict['XGBoost - RandomOverSampler'][k + 1] = [get_scores(y.iloc[test], clf.predict(X.iloc[test])), 
                                                             str(clf.get_params()['estimator']),
                                                             '[' + str(cnf_matrix[0][0]) + ', ' + str(round(cnf_matrix[1][1],2)) + ']']
    
    scores_list.append(roc_auc_score(y.iloc[test], clf.predict(X.iloc[test])))
    
print('Mean: ' + str(np.mean(scores_list)))
print('Std deviation: ' + str(np.std(scores_list)))

Fold: 1
ROC_AUC: 0.6663981958762887
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', RandomOverSampler(random_state=42, ratio=None, return_indices=False,
         sampling_strategy='auto')), ('clf', XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_...ate=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1))])
[Confusion Matrix]: [0.8015463917525774, 0.53]
Fold: 2
ROC_AUC: 0.679546188630491
Estimator and parameters: Pipeline(memory=None,
     steps=[('sampling', RandomOverSampler(random_state=42, ratio=None, return_indices=False,
         sampling_strategy='auto')), ('clf', XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_...ate=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=N

In [69]:
with open('base_Over_wo_wfs.pickle', 'wb') as handle:
    pickle.dump(results_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)